In [1]:
from array import array
import time
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from numpy import random, arange
from scipy.signal import butter, sosfilt, sosfreqz, lfilter
import datetime
import random
from tkinter import *  
from tkinter.ttk import Progressbar  
from tkinter import ttk  
from tkinter import filedialog  
import sqlalchemy
from scipy import interpolate
import sys
import mysql
import mysql.connector
import pickle
import warnings
import xgboost as xgb
import lightgbm as lgbm
import catboost as catb
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
warnings.simplefilter('ignore')
import pathlib
import os



In [2]:
from forces import vagon_1
from forces import vagon_2
from forces import vagon_3
from forces import vagon_4
from tensions import krom_tension
from tensions import f_shpal
from tensions import f_ballast
from tensions import f_ploch


In [3]:
line = pd.read_excel('dannye.xlsx', sheet_name='line')
trains = pd.read_excel('dannye.xlsx', sheet_name='trains')
pod_sos = pd.read_excel('dannye.xlsx', sheet_name='pod_sos')
vsp_konstr = pd.read_excel('dannye.xlsx', sheet_name='vsp_constr')

In [4]:
line

,km,pk,way,vsp_type,vsp_cnd,rail,wear_mm,tie,fasten,hball_sm,rad_m,direct,h_mm,i_prm,l_m,Shkol_mm
0,1,1,1,1,1,P65,1,beton,kb,20,10000,strgh,0,5,50,1520
1,1,1,1,1,1,P65,1,beton,gbr_bp,30,10000,strgh,0,0,50,1525
2,1,2,1,1,1,P75,1,beton,gbr_pod,40,5000,lft,10,-5,50,1515
3,1,2,1,1,1,P65,2,beton,ars,25,10000,strgh,0,0,50,1530
4,1,3,1,1,1,P65,1,beton,pandr,45,10000,strgh,0,0,50,1520
5,1,3,1,1,1,P65,2,beton,whslo,20,2500,rgt,15,4,50,1520
6,1,4,1,2,2,P65,3,der,do,30,10000,strgh,0,0,50,1520
7,1,4,1,2,2,P75,5,der,kd,40,10000,strgh,0,1,50,1520
8,1,5,1,2,2,P65,9,beton,kb,25,350,lft,100,0,50,1520
9,1,5,1,1,1,P65,5,beton,gbr_bp,45,10000,strgh,0,0,50,1520


In [5]:
trains

,mpoezda_mean,poezd_gamma,meankver_force,meankgor_force,meankprd_force,meankram_force,rmskver_force,rmskgor_force,rmskprd_force,rmskram_force
0,1000,0.15,1,1,1,1,1,1,1,1
1,2000,0.15,1,1,1,1,1,1,1,1
2,3000,0.15,1,1,1,1,1,1,1,1
3,4000,0.10,1,1,1,1,1,1,1,1
4,5000,0.10,1,1,1,1,1,1,1,1
5,6000,0.10,1,1,1,1,1,1,1,1
6,7000,0.10,1,1,1,1,1,1,1,1
7,8000,0.15,1,1,1,1,1,1,1,1


In [6]:
pod_sos

,vag_type,p_os,gamma,gamma_20,gamma_40,gamma_60,gamma_80,gamma_100,gamma_120,gamma_140,meankver_force,meankgor_force,meankprd_force,meankram_force,rmskver_force,rmskgor_force,rmskprd_force,rmskram_force
0,1,12,0.1,0.00,0.2,0.2,0.2,0.25,0.10,0.05,1,1,1,1,1,1,1,1
1,2,10,0.2,0.05,0.2,0.2,0.2,0.20,0.15,0.00,1,1,1,1,1,1,1,1
2,3,25,0.4,0.20,0.2,0.4,0.2,0.00,0.00,0.00,1,1,1,1,1,1,1,1
3,4,27,0.3,0.20,0.2,0.5,0.1,0.00,0.00,0.00,1,1,1,1,1,1,1,1
4,5,27,0.0,0.20,0.2,0.5,0.1,0.00,0.00,0.00,1,1,1,1,1,1,1,1


In [7]:
vsp_konstr

,rail,epur,tie,ballast,fasten,proklad,Uv_kgs,kv_sm,kg_sm,lsh_sm,L,wv_cm3,wg_cm3,alfa,wpod_sm2,omega_sm2,b_sm,ksi,hball_sm
0,P75,1840,beton,sheb,all,tipov,1500,0.01438,0.021570,55,0.246,492,89,0.403,518,3092,27.6,0.7,60
1,P75,2000,beton,sheb,all,tipov,1670,0.01477,0.022155,51,0.246,492,89,0.403,518,3092,27.6,0.7,60
2,P65,1840,beton,sheb,all,tipov,1000,0.01338,0.020070,55,0.261,417,75,0.403,518,3092,27.6,0.7,55
3,P65,1840,beton,sheb,kb,tipov,1000,0.01338,0.020070,55,0.261,417,75,0.403,518,3092,27.6,0.7,55
4,P65,1840,beton,sheb,ars,tipov,1450,0.01486,0.022290,55,0.261,417,75,0.403,518,3092,27.6,0.7,55
5,P65,1840,beton,sheb,gbr_bp,tipov,940,0.01334,0.020010,55,0.261,417,75,0.403,518,3092,27.6,0.7,55
6,P65,1840,beton,sheb,gbr_pod,tipov,1020,0.01359,0.020385,55,0.261,417,75,0.403,518,3092,27.6,0.7,55
7,P65,2000,beton,sheb,all,tipov,1100,0.01421,0.021315,51,0.261,417,75,0.403,518,3092,27.6,0.7,55
8,P65,2000,beton,sheb,kb,tipov,1100,0.01421,0.021315,51,0.261,417,75,0.403,518,3092,27.6,0.7,55
9,P65,2000,beton,sheb,ars,tipov,1550,0.01511,0.022665,51,0.261,417,75,0.403,518,3092,27.6,0.7,55


In [8]:
vsp_type	vsp_cstr	vsp_cnd	rad_m	h_mm	i_prm	l_m	Shkol_mm	f_tr
VSP_type = input_val[0] 
    Condition = input_val[1] 
    Radius = input_val[2]
    h = input_val[3]
    V = input_val[4]
    axle_load = input_val[5]
    Sh_Kol = input_val[6] 
    mu_fr = input_val[7] 
    vag_type	p_os	gamma	gamma_20	gamma_40	gamma_60	
    gamma_80	gamma_100	gamma_120	gamma_140	
    meankver_force	meankgor_force	meankprd_force	meankram_force	
    rmskver_force	rmskgor_force	rmskprd_force	rmskram_force


SyntaxError: invalid syntax (<ipython-input-8-2baee07c42d5>, line 1)

In [9]:
# бЛОК вычисления сил для каждого элемента плана с полным учетом вагонопотока на линии
force_line = np.zeros((len(line.index),14))
for index_line in range(len(line.index)):
    #print (index)
    input_val = np.array([0, 0, 0,0,0,0,0,0.00])
    input_val[0]= line['vsp_type'].loc[line['vsp_type'].index == index_line].values
    input_val[1]= line['vsp_cnd'].loc[line['vsp_cnd'].index == index_line].values
    input_val[2]= line['rad_m'].loc[line['rad_m'].index == index_line].values
    input_val[3]= line['h_mm'].loc[line['h_mm'].index == index_line].values
    input_val[6]= line['Shkol_mm'].loc[line['Shkol_mm'].index == index_line].values
    input_val[7]= 0.25 
    force_index = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
    for index_vagon in range(len(pod_sos.index)):
        gamma = pod_sos['gamma'].loc[pod_sos['gamma'].index == index_vagon].values
        p_os = pod_sos['p_os'].loc[pod_sos['p_os'].index == index_vagon].values
        vag_type = pod_sos['vag_type'].loc[pod_sos['vag_type'].index == index_vagon].values  
        meankver_force = pod_sos['meankver_force'].loc[pod_sos['meankver_force'].index == index_vagon].values  
        meankgor_force = pod_sos['meankgor_force'].loc[pod_sos['meankgor_force'].index == index_vagon].values  
        meankprd_force = pod_sos['meankprd_force'].loc[pod_sos['meankprd_force'].index == index_vagon].values  
        meankram_force = pod_sos['meankram_force'].loc[pod_sos['meankram_force'].index == index_vagon].values  
        rmskver_force = pod_sos['rmskver_force'].loc[pod_sos['rmskver_force'].index == index_vagon].values  
        rmskgor_force = pod_sos['rmskgor_force'].loc[pod_sos['rmskgor_force'].index == index_vagon].values  
        rmskprd_force = pod_sos['rmskprd_force'].loc[pod_sos['rmskprd_force'].index == index_vagon].values  
        rmskram_force = pod_sos['rmskram_force'].loc[pod_sos['rmskram_force'].index == index_vagon].values  
        force_v = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
        for v in (10,30,50,70,90,110,130):
            gamma_v_name = 'gamma_' + str(10+v)
            gamma_v = pod_sos[gamma_v_name].loc[pod_sos[gamma_v_name].index == index_vagon].values 
            input_val[4] = v
            input_val[5] = p_os
            #print (input_val)
            #print (vag_type, gamma_v)
            if vag_type == 1:
                force = vagon_1 (input_val, 'return')*gamma_v
                for i in [0,1,2]:
                    force[i] = force[i]*meankver_force
                for i in [3,4,5]:
                    force[i] = force[i]*rmskver_force
                for i in [6,7,8]:
                    force[i] = force[i]*meankgor_force
                for i in [9,10,11]:
                    force[i] = force[i]*rmskgor_force
                force[12] = force[12]*meankram_force
                force[13] = force[13]*rmskram_force
                #print (force)
            elif vag_type == 2:
                force = vagon_2 (input_val, 'return')*gamma_v
                #print (force)
                for i in [0,1,2]:
                    force[i] = force[i]*meankver_force
                for i in [3,4,5]:
                    force[i] = force[i]*rmskver_force
                for i in [6,7,8]:
                    force[i] = force[i]*meankgor_force
                for i in [9,10,11]:
                    force[i] = force[i]*rmskgor_force
                force[12] = force[12]*meankram_force
                force[13] = force[13]*rmskram_force
                
            elif vag_type == 3:
                force = vagon_3 (input_val, 'return')*gamma_v
                #print (force)
                for i in [0,1,2]:
                    force[i] = force[i]*meankver_force
                for i in [3,4,5]:
                    force[i] = force[i]*rmskver_force
                for i in [6,7,8]:
                    force[i] = force[i]*meankgor_force
                for i in [9,10,11]:
                    force[i] = force[i]*rmskgor_force
                force[12] = force[12]*meankram_force
                force[13] = force[13]*rmskram_force
            elif vag_type == 4:
                force = vagon_4 (input_val, 'return')*gamma_v
                #print (force)
                for i in [0,1,2]:
                    force[i] = force[i]*meankver_force
                for i in [3,4,5]:
                    force[i] = force[i]*rmskver_force
                for i in [6,7,8]:
                    force[i] = force[i]*meankgor_force
                for i in [9,10,11]:
                    force[i] = force[i]*rmskgor_force
                force[12] = force[12]*meankram_force
                force[13] = force[13]*rmskram_force
            #else:
                #print ("Номер типа подвижного состава указан неверно или отстуствует в базе")
            force_v = force_v + force
        force_index = force_index +  force_v*gamma 
    for i in range(0,14):
        force_line[index_line, i] = force_index[i]
print (force_line)



[[ 84.8355   93.382    89.10825  10.351    11.3565   10.897    17.067
   31.0465   24.05325   7.2145    8.5125    7.904     2.5315    8.233  ]
 [ 84.8305   93.3625   89.094    10.338    11.3625   10.89     17.0655
   31.0465   24.0525    7.2145    8.512     7.904     2.5355    8.233  ]
 [ 89.3245   93.727    91.52775   4.4115    4.939     4.695     9.553
   19.1385   14.34775   4.807     4.6245    4.7535    3.6575    5.3515 ]
 [ 84.833    93.3575   89.09125  10.368    11.3655   10.909    17.0655
   31.0465   24.0525    7.2155    8.512     7.903     2.5595    8.228  ]
 [ 84.8355   93.382    89.10825  10.351    11.3565   10.897    17.067
   31.0465   24.05325   7.2145    8.5125    7.904     2.5315    8.233  ]
 [112.7925   85.387    99.08425   2.18      2.346     2.2895    6.51
   11.7615    9.13725   5.3065    6.317     5.8725   11.7755    6.3005 ]
 [ 84.9      93.8225   89.35975  11.054    12.114    11.6125   17.1495
   31.0105   24.082     7.8245    9.251     8.599     2.4125    8.955 

In [10]:
# бЛОК вычисления сил для каждого элемента плана с полным учетом поездопотока и уклона на линии
force_poezd = np.zeros((len(line.index),14))
aprox_masxi = [-1.8656*(10**-18), 2.4627*(10**-12), 2.0676*(10**-7),1.0262] # коэфф к апроксимации зависимости сил от массы и уклона
for index_line in range(len(line.index)):
    grad = line['i_prm'].loc[line['i_prm'].index == index_line].values 
    
    force_t = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
    
    for index_poezd in range(len(trains.index)):
        massa_poezd = trains['mpoezda_mean'].loc[trains['mpoezda_mean'].index == index_poezd].values
        gamma_poezd = trains['poezd_gamma'].loc[trains['poezd_gamma'].index == index_poezd].values
        k_massxi = (((grad*massa_poezd)**3)*aprox_masxi[0] + ((grad*massa_poezd)**2)*aprox_masxi[1]\
                     + (grad*massa_poezd)*aprox_masxi[2] + aprox_masxi[3])       
        meankver_force = trains['meankver_force'].loc[trains['meankver_force'].index == index_poezd].values  
        meankgor_force = trains['meankgor_force'].loc[trains['meankgor_force'].index == index_poezd].values  
        meankprd_force = trains['meankprd_force'].loc[trains['meankprd_force'].index == index_poezd].values  
        meankram_force = trains['meankram_force'].loc[trains['meankram_force'].index == index_poezd].values  
        rmskver_force = trains['rmskver_force'].loc[trains['rmskver_force'].index == index_poezd].values  
        rmskgor_force = trains['rmskgor_force'].loc[trains['rmskgor_force'].index == index_poezd].values  
        rmskprd_force = trains['rmskprd_force'].loc[trains['rmskprd_force'].index == index_poezd].values  
        rmskram_force = trains['rmskram_force'].loc[trains['rmskram_force'].index == index_poezd].values
        for i in [0,1,2]:
            force[i] = force_line[index_line][i]*meankver_force*gamma_poezd*k_massxi
        for i in [3,4,5]:
            force[i] = force_line[index_line][i]*rmskver_force*gamma_poezd*k_massxi
        for i in [6,7,8]:
            force[i] = force_line[index_line][i]*meankgor_force*gamma_poezd*k_massxi
        for i in [9,10,11]:
            force[i] = force_line[index_line][i]*rmskgor_force*gamma_poezd*k_massxi 
        force[12] = force_line[index_line][12]*meankram_force*gamma_poezd*k_massxi
        force[13] = force_line[index_line][13]*rmskram_force*gamma_poezd*k_massxi 
        
        
        #print(force)
        force_t = force_t + force 
    #print (force_t)
    for i in range(0,14):
        force_poezd[index_line, i] = force_t[i]
print (force_poezd)


    

    

[[ 87.55912827  96.3800121   91.96905414  10.68331697  11.72109836
   11.2468462   17.61493293  32.04324223  24.82547521   7.44612021
    8.78579226   8.15775648   2.61277335   8.49731897]
 [ 87.0530591   95.8085975   91.4282628   10.6088556   11.6601975
   11.175318    17.5126161   31.8599183   24.6826755    7.4035199
    8.7350144    8.1110848    2.6019301    8.4487046 ]
 [ 91.40463203  95.90965464  93.65918991   4.5142322    5.05401628
    4.80433417   9.77546418  19.58418519  14.68187126   4.91894235
    4.73219241   4.86419648   3.74267353   5.47612232]
 [ 87.0556246   95.8034665   91.42544075  10.6396416   11.6632761
   11.1948158   17.5126161   31.8599183   24.6826755    7.4045461
    8.7350144    8.1100586    2.6265589    8.4435736 ]
 [ 87.0581901   95.8286084   91.44288615  10.6221962   11.6540403
   11.1825014   17.5141554   31.8599183   24.68344515   7.4035199
    8.7355275    8.1110848    2.5978253    8.4487046 ]
 [116.25466852  88.00795603 102.12564345   2.24691515   2.418

In [11]:
# Формирование итогового массива с силами для каждого элемента
forces_itogo = pd.DataFrame(force_poezd)
forces_itogo.columns = ['Mean_F_vertR_kN', 'Mean_F_vertL_kN', 'Mean_F_vert_kN', 'sigma_F_vertR_kN', 'sigma_F_vertL_kN',\
                        'sigma_F_vert_kN','Mean_F_sideR_kN', 'Mean_F_sideL_kN', 'Mean_F_side_kN', 'sigma_F_sideR_kN',\
                        'sigma_F_sideL_kN', 'sigma_F_side_kN', 'Mean_Hp_kN', 'sigma_Hp_kN' ]
forces_itogo

,Mean_F_vertR_kN,Mean_F_vertL_kN,Mean_F_vert_kN,sigma_F_vertR_kN,sigma_F_vertL_kN,sigma_F_vert_kN,Mean_F_sideR_kN,Mean_F_sideL_kN,Mean_F_side_kN,sigma_F_sideR_kN,sigma_F_sideL_kN,sigma_F_side_kN,Mean_Hp_kN,sigma_Hp_kN
0,87.559128,96.380012,91.969054,10.683317,11.721098,11.246846,17.614933,32.043242,24.825475,7.446120,8.785792,8.157756,2.612773,8.497319
1,87.053059,95.808598,91.428263,10.608856,11.660198,11.175318,17.512616,31.859918,24.682676,7.403520,8.735014,8.111085,2.601930,8.448705
2,91.404632,95.909655,93.659190,4.514232,5.054016,4.804334,9.775464,19.584185,14.681871,4.918942,4.732192,4.864196,3.742674,5.476122
3,87.055625,95.803466,91.425441,10.639642,11.663276,11.194816,17.512616,31.859918,24.682676,7.404546,8.735014,8.110059,2.626559,8.443574
4,87.058190,95.828608,91.442886,10.622196,11.654040,11.182501,17.514155,31.859918,24.683445,7.403520,8.735528,8.111085,2.597825,8.448705
5,116.254669,88.007956,102.125643,2.246915,2.418011,2.359776,6.709825,12.122520,9.417718,5.469383,6.510900,6.052757,12.136949,6.493894
6,87.124380,96.280650,91.700975,11.343615,12.431387,11.916748,17.598817,31.822975,24.712948,8.029502,9.493376,8.824294,2.475708,9.189621
7,87.204918,96.369651,91.785744,11.354101,12.442878,11.927763,17.615085,31.852392,24.735793,8.036924,9.502152,8.832451,2.477996,9.198116
8,99.898005,95.909678,97.901789,7.792963,7.857100,7.846838,18.136546,16.422279,17.281465,4.642529,6.494820,5.760574,6.451206,6.031490
9,87.058190,95.828608,91.442886,10.622196,11.654040,11.182501,17.514155,31.859918,24.683445,7.403520,8.735528,8.111085,2.597825,8.448705


In [12]:
# Формирование файла с силами для каждого элемента
force_line = line.join(forces_itogo)
force_line.to_csv('force_line.csv')
force_line

,km,pk,way,vsp_type,vsp_cnd,rail,wear_mm,tie,fasten,hball_sm,...,sigma_F_vertL_kN,sigma_F_vert_kN,Mean_F_sideR_kN,Mean_F_sideL_kN,Mean_F_side_kN,sigma_F_sideR_kN,sigma_F_sideL_kN,sigma_F_side_kN,Mean_Hp_kN,sigma_Hp_kN
0,1,1,1,1,1,P65,1,beton,kb,20,...,11.721098,11.246846,17.614933,32.043242,24.825475,7.446120,8.785792,8.157756,2.612773,8.497319
1,1,1,1,1,1,P65,1,beton,gbr_bp,30,...,11.660198,11.175318,17.512616,31.859918,24.682676,7.403520,8.735014,8.111085,2.601930,8.448705
2,1,2,1,1,1,P75,1,beton,gbr_pod,40,...,5.054016,4.804334,9.775464,19.584185,14.681871,4.918942,4.732192,4.864196,3.742674,5.476122
3,1,2,1,1,1,P65,2,beton,ars,25,...,11.663276,11.194816,17.512616,31.859918,24.682676,7.404546,8.735014,8.110059,2.626559,8.443574
4,1,3,1,1,1,P65,1,beton,pandr,45,...,11.654040,11.182501,17.514155,31.859918,24.683445,7.403520,8.735528,8.111085,2.597825,8.448705
5,1,3,1,1,1,P65,2,beton,whslo,20,...,2.418011,2.359776,6.709825,12.122520,9.417718,5.469383,6.510900,6.052757,12.136949,6.493894
6,1,4,1,2,2,P65,3,der,do,30,...,12.431387,11.916748,17.598817,31.822975,24.712948,8.029502,9.493376,8.824294,2.475708,9.189621
7,1,4,1,2,2,P75,5,der,kd,40,...,12.442878,11.927763,17.615085,31.852392,24.735793,8.036924,9.502152,8.832451,2.477996,9.198116
8,1,5,1,2,2,P65,9,beton,kb,25,...,7.857100,7.846838,18.136546,16.422279,17.281465,4.642529,6.494820,5.760574,6.451206,6.031490
9,1,5,1,1,1,P65,5,beton,gbr_bp,45,...,11.654040,11.182501,17.514155,31.859918,24.683445,7.403520,8.735528,8.111085,2.597825,8.448705


In [13]:
vsp_konstr

,rail,epur,tie,ballast,fasten,proklad,Uv_kgs,kv_sm,kg_sm,lsh_sm,L,wv_cm3,wg_cm3,alfa,wpod_sm2,omega_sm2,b_sm,ksi,hball_sm
0,P75,1840,beton,sheb,all,tipov,1500,0.01438,0.021570,55,0.246,492,89,0.403,518,3092,27.6,0.7,60
1,P75,2000,beton,sheb,all,tipov,1670,0.01477,0.022155,51,0.246,492,89,0.403,518,3092,27.6,0.7,60
2,P65,1840,beton,sheb,all,tipov,1000,0.01338,0.020070,55,0.261,417,75,0.403,518,3092,27.6,0.7,55
3,P65,1840,beton,sheb,kb,tipov,1000,0.01338,0.020070,55,0.261,417,75,0.403,518,3092,27.6,0.7,55
4,P65,1840,beton,sheb,ars,tipov,1450,0.01486,0.022290,55,0.261,417,75,0.403,518,3092,27.6,0.7,55
5,P65,1840,beton,sheb,gbr_bp,tipov,940,0.01334,0.020010,55,0.261,417,75,0.403,518,3092,27.6,0.7,55
6,P65,1840,beton,sheb,gbr_pod,tipov,1020,0.01359,0.020385,55,0.261,417,75,0.403,518,3092,27.6,0.7,55
7,P65,2000,beton,sheb,all,tipov,1100,0.01421,0.021315,51,0.261,417,75,0.403,518,3092,27.6,0.7,55
8,P65,2000,beton,sheb,kb,tipov,1100,0.01421,0.021315,51,0.261,417,75,0.403,518,3092,27.6,0.7,55
9,P65,2000,beton,sheb,ars,tipov,1550,0.01511,0.022665,51,0.261,417,75,0.403,518,3092,27.6,0.7,55


In [14]:
kv
    kg
    wg
    wv
      f_shp

IndentationError: unexpected indent (<ipython-input-14-562ac0871a74>, line 2)

In [38]:
# бЛОК вычисления напряжений в рельсах для каждого элемента линии с полным учетом поездопотока и уклона на линии
#вычисления ведутся для осредненных значений
rail_streses = np.zeros((len(line.index),6))
tie_forces = np.zeros((len(line.index),4))
balast_streses = np.zeros((len(line.index),2))
opzp_streses = np.zeros((len(line.index),2))
for index_line in range(len(line.index)):
    input_val = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
    rail = line['rail'].loc[line['rail'].index == index_line].values 
    tie = line['tie'].loc[line['tie'].index == index_line].values
    fasten = line['fasten'].loc[line['fasten'].index == index_line].values
    input_val[10] = line['hball_sm'].loc[line['hball_sm'].index == index_line].values
    rad_m = line['rad_m'].loc[line['rad_m'].index == index_line].values
     
    if rad_m > 1200:
        epur = 1840
    else:
        epur = 2000
    ballast = 'sheb'
    proklad = 'tipov'
    input_val[6] = 100000/epur
    input_val[11] = 1
   
    vsp_index = vsp_konstr.index[(vsp_konstr['rail'] == rail[0]) & (vsp_konstr['epur'] == epur)\
                                & (vsp_konstr['tie'] == tie[0]) & (vsp_konstr['fasten'] == fasten[0])].tolist()
    if len(vsp_index) == 0:
        vsp_index = vsp_konstr.index[(vsp_konstr['rail'] == rail[0]) & (vsp_konstr['epur'] == epur)\
                                & (vsp_konstr['tie'] == tie[0]) & (vsp_konstr['fasten'] == 'all')].tolist()
    if len(vsp_index) == 0:
        vsp_index = vsp_konstr.index[(vsp_konstr['rail'] == rail[0]) & (vsp_konstr['epur'] == epur)\
                                & (vsp_konstr['tie'] == tie[0])].tolist()
    if len(vsp_index) == 0:
        print ('В базе отсуствует соответствующая конструкция пути')
        print ('Для расчета будет принята типовая конструкция пути - Р65, жб шпалы, щеб, 1840 шт/км, ЖБР-Ш')
        vsp_index = [4]
    if len(vsp_index) > 1:
        vsp_index = [vsp_index[0]] 
    input_val[4] = vsp_konstr['kg_sm'].loc[vsp_konstr['kg_sm'].index == vsp_index].values 
    input_val[5] = vsp_konstr['kv_sm'].loc[vsp_konstr['kv_sm'].index == vsp_index].values 
    input_val[7] = vsp_konstr['wg_cm3'].loc[vsp_konstr['wg_cm3'].index == vsp_index].values 
    input_val[8] = vsp_konstr['wv_cm3'].loc[vsp_konstr['wv_cm3'].index == vsp_index].values  
    
    input_val[9] = vsp_konstr['omega_sm2'].loc[vsp_konstr['omega_sm2'].index == vsp_index].values 
    force_names = ['Mean_F_vert_kN', 'sigma_F_vert_kN', 'Mean_F_side_kN','sigma_F_side_kN']
    i = 0
    for force_name in force_names:
        input_val[i] = force_line[force_name].loc[force_line[force_name].index == index_line].values
        i = i+1
    tensions = krom_tension(input_val)
    tie_force =  f_shpal(input_val) 
    balast_stress = f_ballast(input_val) 
    opzp_stress = f_ploch(input_val)              
    
    for i in range(0,6):
        rail_streses[index_line, i] = tensions[i]
    for i in range(0,4):
        tie_forces[index_line, i] = tie_force[i]
    for i in range(0,2):
        balast_streses[index_line, i] = balast_stress[i] 
    for i in range(0,2):
        opzp_streses[index_line, i] = opzp_stress[i] 
        
   



array([[50.8,  3.7],
       [49.5,  3.6],
       [50.6,  1.9],
       [49.9,  3.7],
       [48.3,  3.7],
       [60.2,  1.3],
       [49.7,  3.9],
       [49. ,  3.9],
       [55.8,  2.7],
       [48.3,  3.7],
       [48.8,  3.7],
       [58.9,  1.6],
       [51.4,  3.8],
       [50.6,  1.9],
       [49.9,  3.7],
       [48.3,  3.7],
       [49.7,  3.9],
       [49. ,  3.9],
       [55.8,  2.7],
       [48.3,  3.7]])

In [29]:
rail_streses_itogo = pd.DataFrame(rail_streses)
rail_streses_itogo.columns = ['Mean_nar_krom_MPa', 'RMS_nar_krom_MPa', 'Mean_vntr_krom_MPa', 'RMS_vntr_krom_MPa',\
                             'Mean_osev_MPa', 'RMS_osev_MPa']

rail_streses = line.join(rail_streses_itogo)
rail_streses.to_csv('rail_streses_line.csv')
rail_streses 

,km,pk,way,vsp_type,vsp_cnd,rail,wear_mm,tie,fasten,hball_sm,...,h_mm,i_prm,l_m,Shkol_mm,Mean_nar_krom_MPa,RMS_nar_krom_MPa,Mean_vntr_krom_MPa,RMS_vntr_krom_MPa,Mean_osev_MPa,RMS_osev_MPa
0,1,1,1,1,1,P65,1,beton,kb,20,...,0,5,50,1520,82.4,14.5,-0.0,14.5,41.2,5.0
1,1,1,1,1,1,P65,1,beton,gbr_bp,30,...,0,0,50,1525,82.2,14.4,-0.0,14.4,41.1,5.0
2,1,2,1,1,1,P75,1,beton,gbr_pod,40,...,10,-5,50,1515,52.2,6.6,14.0,6.6,33.1,1.7
3,1,2,1,1,1,P65,2,beton,ars,25,...,0,0,50,1530,73.8,12.9,-0.0,12.9,36.9,4.5
4,1,3,1,1,1,P65,1,beton,pandr,45,...,0,0,50,1520,82.0,14.4,-0.0,14.4,41.0,5.0
5,1,3,1,1,1,P65,2,beton,whslo,20,...,15,4,50,1520,61.4,10.1,30.1,10.1,45.8,1.1
6,1,4,1,2,2,P65,3,der,do,30,...,0,0,50,1520,109.9,20.9,0.1,20.9,55.0,7.1
7,1,4,1,2,2,P75,5,der,kd,40,...,0,1,50,1520,99.3,18.8,0.3,18.8,49.8,6.5
8,1,5,1,2,2,P65,9,beton,kb,25,...,100,0,50,1520,68.3,9.6,14.3,9.6,41.3,3.3
9,1,5,1,1,1,P65,5,beton,gbr_bp,45,...,0,0,50,1520,82.2,14.4,-0.0,14.4,41.1,5.0


In [30]:
tie_forces_itogo = pd.DataFrame(tie_forces)
tie_forces_itogo.columns = ['mean_F_shpal_vert_kN','sigma_F_shpal_vert_kN', 'mean_F_shpal_side_kN', 'sigma_F_shpal_side_kN']

tie_forces = line.join(tie_forces_itogo)
tie_forces.to_csv('tie_forces_line.csv')
tie_forces 


,km,pk,way,vsp_type,vsp_cnd,rail,wear_mm,tie,fasten,hball_sm,rad_m,direct,h_mm,i_prm,l_m,Shkol_mm,mean_F_shpal_vert_kN,sigma_F_shpal_vert_kN,mean_F_shpal_side_kN,sigma_F_shpal_side_kN
0,1,1,1,1,1,P65,1,beton,kb,20,10000,strgh,0,5,50,1520,33.4,4.1,13.5,4.4
1,1,1,1,1,1,P65,1,beton,gbr_bp,30,10000,strgh,0,0,50,1525,33.1,4.1,13.4,4.4
2,1,2,1,1,1,P75,1,beton,gbr_pod,40,5000,lft,10,-5,50,1515,36.6,1.9,8.6,2.9
3,1,2,1,1,1,P65,2,beton,ars,25,10000,strgh,0,0,50,1530,36.9,4.5,15.0,4.9
4,1,3,1,1,1,P65,1,beton,pandr,45,10000,strgh,0,0,50,1520,33.2,4.1,13.5,4.4
5,1,3,1,1,1,P65,2,beton,whslo,20,2500,rgt,15,4,50,1520,37.1,0.9,5.1,3.3
6,1,4,1,2,2,P65,3,der,do,30,10000,strgh,0,0,50,1520,24.9,3.2,10.1,3.6
7,1,4,1,2,2,P75,5,der,kd,40,10000,strgh,0,1,50,1520,23.3,3.0,9.4,3.4
8,1,5,1,2,2,P65,9,beton,kb,25,350,lft,100,0,50,1520,34.8,2.8,9.2,3.1
9,1,5,1,1,1,P65,5,beton,gbr_bp,45,10000,strgh,0,0,50,1520,33.1,4.1,13.4,4.4


In [37]:
balast_streses_itogo = pd.DataFrame(balast_streses)
balast_streses_itogo.columns = ['mean_F_ballast_kPa', 'sigma_F_ballast_kPa']

balast_streses = line.join(balast_streses_itogo)
balast_streses.to_csv('balast_streses_line.csv')
balast_streses 

,km,pk,way,vsp_type,vsp_cnd,rail,wear_mm,tie,fasten,hball_sm,rad_m,direct,h_mm,i_prm,l_m,Shkol_mm,mean_F_ballast_kPa,sigma_F_ballast_kPa
0,1,1,1,1,1,P65,1,beton,kb,20,10000,strgh,0,5,50,1520,108.1,13.2
1,1,1,1,1,1,P65,1,beton,gbr_bp,30,10000,strgh,0,0,50,1525,107.2,13.1
2,1,2,1,1,1,P75,1,beton,gbr_pod,40,5000,lft,10,-5,50,1515,118.4,6.1
3,1,2,1,1,1,P65,2,beton,ars,25,10000,strgh,0,0,50,1530,119.4,14.6
4,1,3,1,1,1,P65,1,beton,pandr,45,10000,strgh,0,0,50,1520,107.5,13.1
5,1,3,1,1,1,P65,2,beton,whslo,20,2500,rgt,15,4,50,1520,120.1,2.8
6,1,4,1,2,2,P65,3,der,do,30,10000,strgh,0,0,50,1520,87.3,11.4
7,1,4,1,2,2,P75,5,der,kd,40,10000,strgh,0,1,50,1520,81.8,10.6
8,1,5,1,2,2,P65,9,beton,kb,25,350,lft,100,0,50,1520,112.5,9.0
9,1,5,1,1,1,P65,5,beton,gbr_bp,45,10000,strgh,0,0,50,1520,107.2,13.1


In [39]:
opzp_streses_itogo = pd.DataFrame(opzp_streses)
opzp_streses_itogo.columns = ['mean_F_OPZP_kPa', 'sigma_F_OPZP_kPa']

opzp_streses = line.join(opzp_streses_itogo)
opzp_streses.to_csv('opzp_streses_line.csv')
opzp_streses 

,km,pk,way,vsp_type,vsp_cnd,rail,wear_mm,tie,fasten,hball_sm,rad_m,direct,h_mm,i_prm,l_m,Shkol_mm,mean_F_OPZP_kPa,sigma_F_OPZP_kPa
0,1,1,1,1,1,P65,1,beton,kb,20,10000,strgh,0,5,50,1520,50.8,3.7
1,1,1,1,1,1,P65,1,beton,gbr_bp,30,10000,strgh,0,0,50,1525,49.5,3.6
2,1,2,1,1,1,P75,1,beton,gbr_pod,40,5000,lft,10,-5,50,1515,50.6,1.9
3,1,2,1,1,1,P65,2,beton,ars,25,10000,strgh,0,0,50,1530,49.9,3.7
4,1,3,1,1,1,P65,1,beton,pandr,45,10000,strgh,0,0,50,1520,48.3,3.7
5,1,3,1,1,1,P65,2,beton,whslo,20,2500,rgt,15,4,50,1520,60.2,1.3
6,1,4,1,2,2,P65,3,der,do,30,10000,strgh,0,0,50,1520,49.7,3.9
7,1,4,1,2,2,P75,5,der,kd,40,10000,strgh,0,1,50,1520,49.0,3.9
8,1,5,1,2,2,P65,9,beton,kb,25,350,lft,100,0,50,1520,55.8,2.7
9,1,5,1,1,1,P65,5,beton,gbr_bp,45,10000,strgh,0,0,50,1520,48.3,3.7


In [ ]:
['Mean_F_vert_kN', 'sigma_F_vert_kN', 'Mean_F_side_kN','sigma_F_side_kN']
    q_mean = input_val[0] 
    q_rms = input_val[1] 
    Y_mean = input_val[2] 
    Y_rms = input_val[3] 
    Kg = input_val[4] 
    Kv = input_val[5] 
    l_sh = input_val[6]
    Wg = input_val[7] 
    Wv = input_val[8]
    f_shp = input_val[9]
    h_ballast = input_val[10]
    ballast_type = input_val[11]
    
def f_shpal(input):

    '''
       Функция вычисления сил, передающихся на шпалы от рельса
       Исходные данные:
       Q_mean/rms, кН - вертикальная сила (среднее/СКО)
       Y_mean/rms, кН - боковая сила (среднее/СКО)
       Kg, cм^-1 - коэффициент отностительной жесткости рельсовой нити в гориз.плоскости
       Kv, cм^-1 - коэффициент отностительной жесткости рельсовой нити в вертик.плоскости
       l_shp,см - расстояние между осями шпал
       Выходные данные:
       (mean/sigma) F_shpal_vert/side, кН - среднее значение/СКО сил действующая от рельса на шпалу в вертикальной/горизонтальной плоскости
    '''
    Q_mean = input[0]
    Q_rms = input[1]
    Y_mean = input[2]
    Y_rms = input[3]
    Kg = input[4]
    Kv = input[5]
    l_sh = pinput[6] 


In [ ]:
In [5]: df1.merge(df2) # by default, it does an inner join on the common column(s)
Out[5]: 
   x  y  z
0  2  b  4
1  3  c  5

df2 = df1.copy()
with pd.ExcelWriter('output.xlsx') as writer:  
    df1.to_excel(writer, sheet_name='Sheet_name_1')
    df2.to_excel(writer, sheet_name='Sheet_name_2')
    
#ExcelWriter также можно использовать для добавления в существующий файл Excel:

with pd.ExcelWriter('output.xlsx',
                    mode='a') as writer:  
    df.to_excel(writer, sheet_name='Sheet_name_3')
    
b[:2]  # Первые две строки
array([[ 0,  1,  2,  3],
       [10, 11, 12, 13]])
>>> b[1:3, : : ]  # Вторая и третья строки
array([[10, 11, 12, 13],
       [20, 21, 22, 23]])